# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 21 2022 2:56PM,246984,10,MSP214195,"Methapharm, Inc.",Released
1,Sep 21 2022 2:56PM,246984,10,MSP214196,"Methapharm, Inc.",Released
2,Sep 21 2022 2:56PM,246984,10,MSP214197,"Methapharm, Inc.",Released
3,Sep 21 2022 2:56PM,246984,10,MSP214198,"Methapharm, Inc.",Released
4,Sep 21 2022 2:56PM,246984,10,MSP214199,"Methapharm, Inc.",Released
5,Sep 21 2022 2:56PM,246984,10,MSP214200,"Methapharm, Inc.",Released
6,Sep 21 2022 2:56PM,246984,10,MSP214201,"Methapharm, Inc.",Released
7,Sep 21 2022 2:56PM,246984,10,MSP214203,"Methapharm, Inc.",Released
8,Sep 21 2022 2:56PM,246984,10,MSP214204,"Methapharm, Inc.",Released
9,Sep 21 2022 2:56PM,246984,10,MSP214205,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,246978,Released,1
39,246981,Released,1
40,246982,Released,1
41,246983,Released,1
42,246984,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246978,NaN,NaN,1.0
246981,NaN,NaN,1.0
246982,NaN,NaN,1.0
246983,NaN,NaN,1.0
246984,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246872,0.0,0.0,1.0
246878,0.0,1.0,0.0
246887,0.0,0.0,1.0
246904,0.0,3.0,3.0
246905,0.0,0.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246872,0,0,1
246878,0,1,0
246887,0,0,1
246904,0,3,3
246905,0,0,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246872,0,0,1
1,246878,0,1,0
2,246887,0,0,1
3,246904,0,3,3
4,246905,0,0,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246872,,,1
1,246878,,1,
2,246887,,,1
3,246904,,3,3
4,246905,,,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 21 2022 2:56PM,246984,10,"Methapharm, Inc."
10,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc."
11,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc."
12,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix
13,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc."
14,Sep 21 2022 2:33PM,246977,10,ISDIN Corporation
27,Sep 21 2022 2:28PM,246976,10,ISDIN Corporation
44,Sep 21 2022 2:26PM,246974,10,ISDIN Corporation
77,Sep 21 2022 2:24PM,246975,10,Emerginnova
82,Sep 21 2022 2:15PM,246973,16,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 21 2022 2:56PM,246984,10,"Methapharm, Inc.",,,10
1,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc.",,,1
2,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc.",,,1
3,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix,,,1
4,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc.",,,1
5,Sep 21 2022 2:33PM,246977,10,ISDIN Corporation,,,13
6,Sep 21 2022 2:28PM,246976,10,ISDIN Corporation,,,17
7,Sep 21 2022 2:26PM,246974,10,ISDIN Corporation,,,33
8,Sep 21 2022 2:24PM,246975,10,Emerginnova,,,5
9,Sep 21 2022 2:15PM,246973,16,Sartorius Stedim North America,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 2:56PM,246984,10,"Methapharm, Inc.",10,,
1,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc.",1,,
2,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc.",1,,
3,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix,1,,
4,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc.",1,,
5,Sep 21 2022 2:33PM,246977,10,ISDIN Corporation,13,,
6,Sep 21 2022 2:28PM,246976,10,ISDIN Corporation,17,,
7,Sep 21 2022 2:26PM,246974,10,ISDIN Corporation,33,,
8,Sep 21 2022 2:24PM,246975,10,Emerginnova,5,,
9,Sep 21 2022 2:15PM,246973,16,Sartorius Stedim North America,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 2:56PM,246984,10,"Methapharm, Inc.",10,,
1,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc.",1,,
2,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc.",1,,
3,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix,1,,
4,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 2:56PM,246984,10,"Methapharm, Inc.",10.0,NaN,NaN
1,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
2,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix,1.0,NaN,NaN
4,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 2:56PM,246984,10,"Methapharm, Inc.",10.0,0.0,0.0
1,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
2,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix,1.0,0.0,0.0
4,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4198158,131.0,20.0,3.0
12,493811,2.0,0.0,0.0
15,493855,10.0,0.0,18.0
16,493881,0.0,6.0,0.0
19,1234793,30.0,6.0,0.0
20,1234706,16.0,7.0,0.0
21,246878,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4198158,131.0,20.0,3.0
1,12,493811,2.0,0.0,0.0
2,15,493855,10.0,0.0,18.0
3,16,493881,0.0,6.0,0.0
4,19,1234793,30.0,6.0,0.0
5,20,1234706,16.0,7.0,0.0
6,21,246878,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,131.0,20.0,3.0
1,12,2.0,0.0,0.0
2,15,10.0,0.0,18.0
3,16,0.0,6.0,0.0
4,19,30.0,6.0,0.0
5,20,16.0,7.0,0.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,131.0
1,12,Released,2.0
2,15,Released,10.0
3,16,Released,0.0
4,19,Released,30.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,3.0,0.0,18.0,0.0,0.0,0.0,0.0
Executing,20.0,0.0,0.0,6.0,6.0,7.0,1.0
Released,131.0,2.0,10.0,0.0,30.0,16.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,3.0,0.0,18.0,0.0,0.0,0.0,0.0
1,Executing,20.0,0.0,0.0,6.0,6.0,7.0,1.0
2,Released,131.0,2.0,10.0,0.0,30.0,16.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,3.0,0.0,18.0,0.0,0.0,0.0,0.0
1,Executing,20.0,0.0,0.0,6.0,6.0,7.0,1.0
2,Released,131.0,2.0,10.0,0.0,30.0,16.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()